<a href="https://colab.research.google.com/github/Swagat1342/Enhanced-Stock-Prediction-System-with-Risk-Management/blob/main/finance_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas yfinance scikit-learn tensorflow plotly

In [ ]:
pip install alpha_vantage finnhub-python polygon-api-client

In [1]:
pip install --upgrade yfinance


In [2]:
pip install yfinance ta pandas numpy plotly


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5e91028b6c89f96b58e4d30f4929a7e1b05159d33b156fb78c6dc882ef93194b
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
# ====== Standard Python & Data Libraries ======
import requests                 # For HTTP requests, useful for APIs or custom data fetching
from requests.adapters import HTTPAdapter       # To configure retry strategy for HTTP requests
from urllib3.util.retry import Retry             # Retry logic for stable HTTP connections

import pandas as pd             # Data manipulation and analysis
import numpy as np              # Numerical computing with arrays

from datetime import datetime, timedelta    # Date/time utilities

# ====== Financial Data and ML Libraries ======
import yfinance as yf           # Yahoo Finance API for stock data

from sklearn.preprocessing import StandardScaler, RobustScaler  # Feature scaling for ML
from sklearn.feature_selection import mutual_info_regression, SelectKBest  # Feature selection

# ====== Visualization Libraries ======
import plotly.graph_objects as go    # For interactive plotting
from plotly.subplots import make_subplots  # For creating subplot layouts

# ====== Technical Analysis ======
import ta                        # Technical indicators library

# ====== Other Settings ======
import warnings                  # Handle warnings gracefully
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output


In [4]:
# ======================================
# 🔑 API Keys Configuration
# ======================================

API_KEYS = {
    'ALPHA_VANTAGE': "YOUR_ALPHA_VANTAGE_API_KEY",  # API key for Alpha Vantage service
    'IEX_CLOUD': "YOUR_IEX_CLOUD_API_KEY",          # API key for IEX Cloud service
    'FINNHUB': "YOUR_FINNHUB_API_KEY",              # API key for Finnhub service
    'POLYGON': "YOUR_POLYGON_API_KEY"               # API key for Polygon.io service
}



In [5]:
# ======================================
# 📊 Stock Data Fetcher Class
# ======================================
class StockDataFetcher:
    def __init__(self, api_keys=None):
        # Initialize the fetcher with provided API keys or defaults
        self.api_keys = api_keys or API_KEYS
        # Create a session with retry mechanism to handle temporary HTTP failures
        self.session = self._make_session()
        # Keep track of the last successful data source used
        self.last_source = None

    def _make_session(self):
        # Create a requests session with retry logic for robust HTTP requests
        session = requests.Session()
        retries = Retry(total=3,                    # Retry up to 3 times
                        backoff_factor=1,           # Wait time multiplier between retries
                        status_forcelist=[429, 500, 502, 503, 504])  # Retry on these HTTP errors
        session.mount('https://', HTTPAdapter(max_retries=retries))
        return session

    def fetch_alpha_vantage(self, symbol, interval='5min', outputsize='compact'):
        """Fetch stock data from Alpha Vantage API with time series intraday."""
        print(f"\n🔹 Trying Alpha Vantage for {symbol}...")
        try:
            # Form the URL for Alpha Vantage intraday API
            url = (
                f"https://www.alphavantage.co/query?"
                f"function=TIME_SERIES_INTRADAY&symbol={symbol}"
                f"&interval={interval}&apikey={self.api_keys['ALPHA_VANTAGE']}"
                f"&outputsize={outputsize}"
            )
            r = self.session.get(url, timeout=10)  # Send GET request with timeout
            data = r.json()  # Parse JSON response

            ts_key = f"Time Series ({interval})"  # Key that contains time series data

            # If expected data key is missing, print error note or message
            if ts_key not in data:
                print(f"❌ Alpha Vantage Error: {data.get('Note', data.get('Error Message', 'Unknown'))}")
                return None

            # Convert time series dictionary to DataFrame and set datetime index
            df = pd.DataFrame.from_dict(data[ts_key], orient="index").astype(float)
            df.index = pd.to_datetime(df.index)

            # Rename columns to standard format
            df.rename(columns={
                "1. open": "open", "2. high": "high",
                "3. low": "low", "4. close": "close",
                "5. volume": "volume"
            }, inplace=True)

            print(f"✅ Alpha Vantage Success — {len(df)} records")
            self.last_source = "Alpha Vantage"
            return df.sort_index()  # Make sure data is in chronological order
        except Exception as e:
            print(f"❌ Alpha Vantage failed: {e}")
            return None

    def fetch_polygon(self, symbol, start_date=None, end_date=None, timespan='minute', multiplier=5):
        """Fetch stock data from Polygon.io aggregates endpoint."""
        print(f"\n🔹 Trying Polygon.io for {symbol}...")
        try:
            # Default to last 5 days if dates not provided
            if not start_date:
                start_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
            if not end_date:
                end_date = datetime.now().strftime('%Y-%m-%d')

            # Construct the URL with range parameters
            url = (
                f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/"
                f"{multiplier}/{timespan}/{start_date}/{end_date}?"
                f"apiKey={self.api_keys['POLYGON']}"
            )
            r = self.session.get(url, timeout=10)
            data = r.json()

            # Check if 'results' key exists in JSON response
            if "results" not in data:
                print(f"❌ Polygon.io Error: {data.get('error', 'No results')}")
                return None

            # Extract results and build DataFrame
            df = pd.DataFrame(data["results"])

            # Convert timestamp from milliseconds to datetime
            df["t"] = pd.to_datetime(df["t"], unit="ms")
            df.rename(columns={
                "t": "timestamp", "o": "open", "h": "high",
                "l": "low", "c": "close", "v": "volume"
            }, inplace=True)

            # Set timestamp as index for time series processing
            df.set_index("timestamp", inplace=True)
            print(f"✅ Polygon.io Success — {len(df)} records")
            self.last_source = "Polygon.io"
            return df
        except Exception as e:
            print(f"❌ Polygon.io failed: {e}")
            return None

    def fetch_finnhub(self, symbol):
        """Fetch current stock quote from Finnhub API."""
        print(f"\n🔹 Trying Finnhub for {symbol}...")
        try:
            url = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={self.api_keys['FINNHUB']}"
            r = self.session.get(url, timeout=10)
            data = r.json()

            # Validate if 'c' (current price) in response is valid and nonzero
            if "c" not in data or data.get("c") == 0:
                print(f"❌ Finnhub Error: No data available")
                return None

            # Create a single-row DataFrame with quote data and current timestamp index
            df = pd.DataFrame([{
                "open": data.get("o"),
                "high": data.get("h"),
                "low": data.get("l"),
                "close": data.get("c"),
                "volume": None  # Volume is not provided by this endpoint
            }], index=[datetime.now()])

            print(f"✅ Finnhub Success — Current ${data.get('c')}")
            self.last_source = "Finnhub"
            return df
        except Exception as e:
            print(f"❌ Finnhub failed: {e}")
            return None

    def fetch_yfinance(self, symbol, period="5d", interval="5m"):
        """Fallback fetch stock data from Yahoo Finance."""
        print(f"\n🔹 Trying Yahoo Finance for {symbol}...")
        try:
            df = yf.download(symbol, period=period, interval=interval, progress=False)

            if df.empty:
                print("❌ Yahoo Finance returned no data")
                return None

            # For multi-level columns (e.g., multi-ticker), drop the extra level
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(1)

            # Lowercase all column names for consistency
            df.columns = [col.lower() if isinstance(col, str) else col[0].lower() for col in df.columns]

            print(f"✅ Yahoo Finance Success — {len(df)} records")
            self.last_source = "Yahoo Finance"
            return df
        except Exception as e:
            print(f"❌ Yahoo Finance failed: {e}")
            return None

    def get_stock_data(self, symbol, prefer_source=None):
        """
        Fetch stock data using preferred source or fallback through all APIs.

        Args:
            symbol (str): Stock ticker symbol.
            prefer_source (str): Preferred data source ('alpha_vantage', 'polygon', 'finnhub', or 'yfinance').

        Returns:
            pd.DataFrame or None: Stock data DataFrame or None if all fail.
        """
        # Dictionary mapping source names to fetcher methods
        fetchers = {
            'alpha_vantage': self.fetch_alpha_vantage,
            'polygon': self.fetch_polygon,
            'finnhub': self.fetch_finnhub,
            'yfinance': self.fetch_yfinance
        }

        # Attempt preferred source first if specified
        if prefer_source and prefer_source in fetchers:
            df = fetchers[prefer_source](symbol)
            if df is not None and not df.empty:
                return df

        # Otherwise, try all sources in order excluding preferred source if already tried
        for name, fetcher in fetchers.items():
            if prefer_source == name:  # Skip already tried preferred source
                continue
            df = fetcher(symbol)
            if df is not None and not df.empty:
                print(f"\n✅ Data fetched successfully from: {self.last_source}")
                return df

        # All sources failed, prompt user to check credentials and connectivity
        print("❌ All APIs failed — please check keys and connection.")
        return None


In [6]:
# Instantiate the StockDataFetcher with your API keys dictionary
fetcher = StockDataFetcher(api_keys=API_KEYS)

# Fetch data for SBI stock symbol, optionally specify preferred data source
symbol = 'SBIN.NS'  # NSE ticker for State Bank of India
df_stock = fetcher.get_stock_data(symbol, prefer_source='alpha_vantage')

# Check if data was fetched successfully
if df_stock is not None:
    # Add technical indicators to fetched stock data
    df_enriched = add_technical_indicators(df_stock)

    # Print a preview of the enriched data
    print(df_enriched.head())

    # Generate and show the comprehensive technical analysis Plotly chart
    fig = plot_technical_analysis(df_enriched, ticker_name=symbol)
    fig.show()

else:
    print("Failed to fetch stock data for symbol:", symbol)



🔹 Trying Alpha Vantage for SBIN.NS...
❌ Alpha Vantage Error: Unknown

🔹 Trying Polygon.io for SBIN.NS...
❌ Polygon.io Error: Unknown API Key

🔹 Trying Finnhub for SBIN.NS...
❌ Finnhub Error: No data available

🔹 Trying Yahoo Finance for SBIN.NS...
✅ Yahoo Finance Success — 300 records

✅ Data fetched successfully from: Yahoo Finance


NameError: name 'add_technical_indicators' is not defined

In [7]:
def add_technical_indicators(df):
    """
    Add a comprehensive set of technical indicators to a stock OHLCV DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing columns 'Open', 'High', 'Low', 'Close', 'Volume'.

    Returns:
    pd.DataFrame: DataFrame enriched with multiple technical indicator columns.
    """

    # ========== TREND INDICATORS ==========

    # Calculate Simple Moving Averages (SMA) to identify price trend smoothing
    df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['SMA_50'] = ta.trend.sma_indicator(df['Close'], window=50)
    df['SMA_100'] = ta.trend.sma_indicator(df['Close'], window=100)
    df['SMA_200'] = ta.trend.sma_indicator(df['Close'], window=200)

    # Calculate Exponential Moving Averages (EMA) - reacts faster to recent price changes
    df['EMA_12'] = ta.trend.ema_indicator(df['Close'], window=12)
    df['EMA_20'] = ta.trend.ema_indicator(df['Close'], window=20)
    df['EMA_26'] = ta.trend.ema_indicator(df['Close'], window=26)
    df['EMA_50'] = ta.trend.ema_indicator(df['Close'], window=50)

    # MACD (Moving Average Convergence Divergence) for momentum and trend strength analysis
    df['MACD'] = ta.trend.macd(df['Close'])
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])
    df['MACD_Hist'] = ta.trend.macd_diff(df['Close'])

    # Average Directional Index (ADX) - measures trend strength (non-directional)
    df['ADX'] = ta.trend.adx(df['High'], df['Low'], df['Close'], window=14)
    df['ADX_Pos'] = ta.trend.adx_pos(df['High'], df['Low'], df['Close'], window=14)
    df['ADX_Neg'] = ta.trend.adx_neg(df['High'], df['Low'], df['Close'], window=14)

    # Use a 20-period SMA as a simple trend line proxy
    df['Trend_Line'] = ta.trend.sma_indicator(df['Close'], window=20)

    # ========== MOMENTUM INDICATORS ==========

    # Relative Strength Index (RSI) over 14 and 7 periods to gauge overbought/oversold momentum
    df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)
    df['RSI_7'] = ta.momentum.rsi(df['Close'], window=7)

    # Stochastic Oscillator components (%K and %D) to measure momentum changes
    df['Stoch_K'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], window=14)
    df['Stoch_D'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], window=14)

    # ========== DeMarker Indicator: Custom Implementation ==========

    def demarker(high, low, period=14):
        demax = high.diff()
        demax[demax < 0] = 0
        demin = -low.diff()
        demin[demin < 0] = 0
        demax_ma = demax.rolling(window=period).mean()
        demin_ma = demin.rolling(window=period).mean()
        return demax_ma / (demax_ma + demin_ma)

    df['DeMarker'] = demarker(df['High'], df['Low'], period=14)

    # ========== VOLATILITY INDICATORS ==========

    df['ATR_14'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=14)
    df['ATR_7'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=7)

    bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Mid'] = bb.bollinger_mavg()
    df['BB_Low'] = bb.bollinger_lband()
    df['BB_Width'] = bb.bollinger_wband()
    df['BB_Pct'] = bb.bollinger_pband()

    df['Std_Dev_20'] = df['Close'].rolling(window=20).std()
    df['Std_Dev_50'] = df['Close'].rolling(window=50).std()

    # ========== VOLUME INDICATORS ==========

    df['OBV'] = ta.volume.on_balance_volume(df['Close'], df['Volume'])
    df['ADI'] = ta.volume.acc_dist_index(df['High'], df['Low'], df['Close'], df['Volume'])
    df['AD_Line'] = ta.volume.acc_dist_index(df['High'], df['Low'], df['Close'], df['Volume'])

    # ========== SUPPORT/RESISTANCE & PIVOT POINTS ==========

    df['Pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['R1'] = 2 * df['Pivot'] - df['Low']
    df['S1'] = 2 * df['Pivot'] - df['High']
    df['R2'] = df['Pivot'] + (df['High'] - df['Low'])
    df['S2'] = df['Pivot'] - (df['High'] - df['Low'])
    df['R3'] = df['High'] + 2 * (df['Pivot'] - df['Low'])
    df['S3'] = df['Low'] - 2 * (df['High'] - df['Pivot'])

    df['Resistance_20'] = df['High'].rolling(window=20).max()
    df['Support_20'] = df['Low'].rolling(window=20).min()
    df['Resistance_50'] = df['High'].rolling(window=50).max()
    df['Support_50'] = df['Low'].rolling(window=50).min()

    # ========== MARKET PROFILE (Volume Profile Approximation) ==========

    df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
    df['VWMA_20'] = (df['Close'] * df['Volume']).rolling(window=20).sum() / df['Volume'].rolling(window=20).sum()

    # Remove NaNs caused by rolling calculations
    df.dropna(inplace=True)

    return df


In [8]:
import numpy as np
import pandas as pd

class AdvancedFeatureEngineer:
    """Full-featured technical indicator generator for trading analysis"""

    def __init__(self):
        self.feature_names = []  # Store names of generated features if needed

    # =============================================================
    # 📌 MASTER FEATURE FUNCTION: Adds all indicators to DataFrame
    # =============================================================
    def add_all_features(self, df):
        df = df.copy()  # Work on a copy to avoid modifying original DataFrame

        # Extract key price/volume series for easy reference
        close, high, low, volume = df['close'], df['high'], df['low'], df['volume']

        # Sequentially add a comprehensive list of technical indicators
        df = self._add_sma_ema(df)                     # Moving averages
        df = self._add_macd(df)                         # MACD momentum
        df = self._add_rsi(df)                          # RSI momentum
        df = self._add_stochastic(df)                   # Stochastic oscillator
        df = self._add_adx(df)                          # Average Directional Index (trend strength)
        df = self._add_demarker(df)                     # DeMarker indicator (trend/momentum)
        df = self._add_atr(df)                          # Average True Range (volatility)
        df = self._add_bollinger(df)                    # Bollinger Bands (volatility)
        df = self._add_std_features(df)                 # Rolling standard deviations
        df = self._add_obv(df)                          # On-Balance Volume (volume momentum)
        df = self._add_accumulation_distribution(df)   # Accumulation/Distribution line (volume flow)
        df = self._add_pivot_points(df)                 # Support/resistance pivot levels
        df = self._add_trendlines(df)                   # Trendline slopes (linear regression)
        df = self._add_market_profile(df)               # Market profile volume clusters
        df = self._add_support_resistance(df)           # Dynamic support/resistance levels

        # Drop rows with NaN values caused by rolling calculations
        df.dropna(inplace=True)
        return df

    # =============================================================
    # 📊 SIMPLE MOVING AVERAGES (SMA) / EXPONENTIAL MOVING AVERAGES (EMA)
    # =============================================================
    def _add_sma_ema(self, df):
        for period in [5, 10, 20, 50, 100, 200]:
            # Calculate SMA for given period
            df[f'sma_{period}'] = df['close'].rolling(period).mean()
            # Calculate EMA for given period
            df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
        return df

    # =============================================================
    # 💹 MACD (Moving Average Convergence Divergence)
    # =============================================================
    def _add_macd(self, df):
        exp1 = df['close'].ewm(span=12, adjust=False).mean()  # Fast EMA
        exp2 = df['close'].ewm(span=26, adjust=False).mean()  # Slow EMA
        df['macd'] = exp1 - exp2                              # MACD line
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()  # Signal line
        df['macd_hist'] = df['macd'] - df['macd_signal']     # Histogram
        return df

    # =============================================================
    # 💪 RSI (Relative Strength Index)
    # =============================================================
    def _add_rsi(self, df, period=14):
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(period).mean()  # Average gains
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()  # Average losses
        rs = gain / (loss + 1e-10)                                 # Relative strength
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))              # RSI formula
        return df

    # =============================================================
    # 🎯 STOCHASTIC OSCILLATOR
    # =============================================================
    def _add_stochastic(self, df, period=14):
        low_min = df['low'].rolling(period).min()
        high_max = df['high'].rolling(period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)  # %K line
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(3).mean()                      # %D line (smoothed)
        return df

    # =============================================================
    # 📏 ADX (Average Directional Index) — Trend Strength Indicator
    # =============================================================
    def _add_adx(self, df, period=14):
        high, low, close = df['high'], df['low'], df['close']
        plus_dm = high.diff()
        minus_dm = -low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm < 0] = 0
        tr = self._calculate_atr(df, period)               # True Range for normalization
        plus_di = 100 * (plus_dm.rolling(period).mean() / tr)  # +DI percentage
        minus_di = 100 * (minus_dm.rolling(period).mean() / tr)  # -DI percentage
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)  # Directional index
        df['adx'] = dx.rolling(period).mean()  # Smoothed ADX
        return df

    # Helper function: ATR calculation
    def _calculate_atr(self, df, period=14):
        tr1 = df['high'] - df['low']
        tr2 = abs(df['high'] - df['close'].shift())
        tr3 = abs(df['low'] - df['close'].shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(period).mean()

    # =============================================================
    # 🧭 DEMARKER INDICATOR — Buying/Selling Pressure
    # =============================================================
    def _add_demarker(self, df, period=14):
        demax = df['high'].diff()
        demin = df['low'].diff()
        demax = np.where(demax > 0, demax, 0)  # Only positive moves
        demin = np.where(demin < 0, abs(demin), 0)  # Only negative moves as positive values
        demax_sma = pd.Series(demax).rolling(period).mean()
        demin_sma = pd.Series(demin).rolling(period).mean()
        df['demarker'] = demax_sma / (demax_sma + demin_sma + 1e-10)
        return df

    # =============================================================
    # 📉 ATR (Average True Range) — Market Volatility
    # =============================================================
    def _add_atr(self, df, period=14):
        df['atr'] = self._calculate_atr(df, period)
        return df

    # =============================================================
    # 🎢 BOLLINGER BANDS — Volatility Channel
    # =============================================================
    def _add_bollinger(self, df, period=20):
        sma = df['close'].rolling(period).mean()
        std = df['close'].rolling(period).std()
        df['bb_upper'] = sma + (2 * std)
        df['bb_lower'] = sma - (2 * std)
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / sma
        return df

    # =============================================================
    # 🧮 STANDARD DEVIATION FEATURES — Supplement Volatility
    # =============================================================
    def _add_std_features(self, df):
        for period in [10, 20, 50]:
            df[f'std_{period}'] = df['close'].rolling(period).std()
        return df

    # =============================================================
    # 💰 ON-BALANCE VOLUME (OBV) — Volume Momentum Indicator
    # =============================================================
    def _add_obv(self, df):
        obv = [0]  # Initialize OBV list with zero
        for i in range(1, len(df)):
            if df['close'][i] > df['close'][i-1]:
                obv.append(obv[-1] + df['volume'][i])  # Add volume when price rises
            elif df['close'][i] < df['close'][i-1]:
                obv.append(obv[-1] - df['volume'][i])  # Subtract volume when price falls
            else:
                obv.append(obv[-1])  # No change when price is unchanged
        df['obv'] = obv
        return df

    # =============================================================
    # 🧩 ACCUMULATION/DISTRIBUTION LINE — Volume Flow Indicator
    # =============================================================
    def _add_accumulation_distribution(self, df):
        mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'] + 1e-10)  # Money Flow Multiplier
        df['ad_line'] = (mfm * df['volume']).cumsum()  # Cumulative volume flow
        return df

    # =============================================================
    # 🪜 PIVOT POINTS — Support/Resistance Midpoints
    # =============================================================
    def _add_pivot_points(self, df):
        df['pivot'] = (df['high'] + df['low'] + df['close']) / 3
        df['r1'] = 2 * df['pivot'] - df['low']  # Resistance 1
        df['s1'] = 2 * df['pivot'] - df['high']  # Support 1
        df['r2'] = df['pivot'] + (df['high'] - df['low'])  # Resistance 2
        df['s2'] = df['pivot'] - (df['high'] - df['low'])  # Support 2
        return df

    # =============================================================
    # 📈 TRENDLINES (Slope of Linear Regression over Rolling Window)
    # =============================================================
    def _add_trendlines(self, df, period=20):
        # For each rolling window, calculate slope of linear regression as trend strength proxy
        df['trend_slope'] = df['close'].rolling(period).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True)
        return df

    # =============================================================
    # 🏗️ MARKET PROFILE (Volume Concentrations by Price Bin)
    # =============================================================
    def _add_market_profile(self, df, bins=20):
        # Bin closing prices into quantile bins
        df['price_bin'] = pd.qcut(df['close'], bins, duplicates='drop')
        # Sum volumes by price bin to estimate volume density at price levels
        volume_profile = df.groupby('price_bin')['volume'].sum()
        # Map sum of volume per bin back to each row
        df['market_profile_strength'] = df['price_bin'].map(volume_profile)
        return df

    # =============================================================
    # 🧭 SUPPORT/RESISTANCE LEVELS (Rolling Local Min/Max & Price Position)
    # =============================================================
    def _add_support_resistance(self, df, window=10):
        # Rolling minimum low as support level
        df['support'] = df['low'].rolling(window).min()
        # Rolling maximum high as resistance level
        df['resistance'] = df['high'].rolling(window).max()
        # Relative close price position between support and resistance (0 - support, 1 - resistance)
        df['price_position'] = (df['close'] - df['support']) / (df['resistance'] - df['support'] + 1e-10)
        return df


In [9]:

class AdvancedFeatureEngineer:
    """Full-featured technical indicator generator for trading analysis"""

    def __init__(self):
        # Optionally store feature names here for future reference (not used now)
        self.feature_names = []


In [10]:
    # =============================================================
    # 📌 MASTER FEATURE FUNCTION: Adds all indicators to a DataFrame
    # =============================================================
    def add_all_features(self, df):
        df = df.copy()  # Work on copy to avoid side effects

        # Extract price/volume series for convenience
        close, high, low, volume = df['close'], df['high'], df['low'], df['volume']

        # Add technical features one by one by calling dedicated functions
        df = self._add_sma_ema(df)
        df = self._add_macd(df)
        df = self._add_rsi(df)
        df = self._add_stochastic(df)
        df = self._add_adx(df)
        df = self._add_demarker(df)
        df = self._add_atr(df)
        df = self._add_bollinger(df)
        df = self._add_std_features(df)
        df = self._add_obv(df)
        df = self._add_accumulation_distribution(df)
        df = self._add_pivot_points(df)
        df = self._add_trendlines(df)
        df = self._add_market_profile(df)
        df = self._add_support_resistance(df)

        # Drop rows with NaN created by rolling/window calculations
        df.dropna(inplace=True)
        return df


    # =============================================================
    # 📊 SIMPLE MOVING AVERAGES (SMA) and EXPONENTIAL MOVING AVERAGES (EMA)
    # =============================================================
    def _add_sma_ema(self, df):
        for period in [5, 10, 20, 50, 100, 200]:
            df[f'sma_{period}'] = df['close'].rolling(window=period).mean()
            df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
        return df


In [11]:
    # =============================================================
    # 💹 MACD (Moving Average Convergence Divergence)
    # =============================================================
    def _add_macd(self, df):
        exp1 = df['close'].ewm(span=12, adjust=False).mean()
        exp2 = df['close'].ewm(span=26, adjust=False).mean()
        df['macd'] = exp1 - exp2
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
        df['macd_hist'] = df['macd'] - df['macd_signal']
        return df


    # =============================================================
    # 💪 RSI (Relative Strength Index)
    # =============================================================
    def _add_rsi(self, df, period=14):
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
        return df


In [12]:
    # =============================================================
    # 🎯 STOCHASTIC OSCILLATOR
    # =============================================================
    def _add_stochastic(self, df, period=14):
        low_min = df['low'].rolling(window=period).min()
        high_max = df['high'].rolling(window=period).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(window=3).mean()
        return df


    # =============================================================
    # 📏 ADX (Average Directional Index)
    # =============================================================
    def _add_adx(self, df, period=14):
        high, low, close = df['high'], df['low'], df['close']
        plus_dm = high.diff()
        minus_dm = -low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm < 0] = 0
        tr = self._calculate_atr(df, period)
        plus_di = 100 * (plus_dm.rolling(window=period).mean() / tr)
        minus_di = 100 * (minus_dm.rolling(window=period).mean() / tr)
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)
        df['adx'] = dx.rolling(window=period).mean()
        return df


    # Helper function to calculate ATR used by ADX
    def _calculate_atr(self, df, period=14):
        tr1 = df['high'] - df['low']
        tr2 = abs(df['high'] - df['close'].shift())
        tr3 = abs(df['low'] - df['close'].shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(window=period).mean()


In [13]:
    # =============================================================
    # 🧭 DEMARKER INDICATOR
    # =============================================================
    def _add_demarker(self, df, period=14):
        demax = df['high'].diff()
        demin = df['low'].diff()
        demax = np.where(demax > 0, demax, 0)
        demin = np.where(demin < 0, abs(demin), 0)
        demax_sma = pd.Series(demax).rolling(window=period).mean()
        demin_sma = pd.Series(demin).rolling(window=period).mean()
        df['demarker'] = demax_sma / (demax_sma + demin_sma + 1e-10)
        return df


    # =============================================================
    # 📉 ATR (Average True Range)
    # =============================================================
    def _add_atr(self, df, period=14):
        df['atr'] = self._calculate_atr(df, period)
        return df


    # =============================================================
    # 🎢 BOLLINGER BANDS
    # =============================================================
    def _add_bollinger(self, df, period=20):
        sma = df['close'].rolling(window=period).mean()
        std = df['close'].rolling(window=period).std()
        df['bb_upper'] = sma + (2 * std)
        df['bb_lower'] = sma - (2 * std)
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / sma
        return df


In [14]:
    # =============================================================
    # 🧮 STANDARD DEVIATION FEATURES
    # =============================================================
    def _add_std_features(self, df):
        for period in [10, 20, 50]:
            df[f'std_{period}'] = df['close'].rolling(window=period).std()
        return df


    # =============================================================
    # 💰 ON-BALANCE VOLUME (OBV)
    # =============================================================
    def _add_obv(self, df):
        obv = [0]
        for i in range(1, len(df)):
            if df['close'][i] > df['close'][i-1]:
                obv.append(obv[-1] + df['volume'][i])
            elif df['close'][i] < df['close'][i-1]:
                obv.append(obv[-1] - df['volume'][i])
            else:
                obv.append(obv[-1])
        df['obv'] = obv
        return df


    # =============================================================
    # 🧩 ACCUMULATION/DISTRIBUTION LINE
    # =============================================================
    def _add_accumulation_distribution(self, df):
        mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'] + 1e-10)
        df['ad_line'] = (mfm * df['volume']).cumsum()
        return df


In [15]:
    # =============================================================
    # 🪜 PIVOT POINTS
    # =============================================================
    def _add_pivot_points(self, df):
        df['pivot'] = (df['high'] + df['low'] + df['close']) / 3
        df['r1'] = 2 * df['pivot'] - df['low']
        df['s1'] = 2 * df['pivot'] - df['high']
        df['r2'] = df['pivot'] + (df['high'] - df['low'])
        df['s2'] = df['pivot'] - (df['high'] - df['low'])
        return df


    # =============================================================
    # 📈 TRENDLINES (linear regression slope)
    # =============================================================
    def _add_trendlines(self, df, period=20):
        df['trend_slope'] = df['close'].rolling(window=period).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True)
        return df


    # =============================================================
    # 🏗️ MARKET PROFILE (volume by price bins)
    # =============================================================
    def _add_market_profile(self, df, bins=20):
        df['price_bin'] = pd.qcut(df['close'], q=bins, duplicates='drop')
        volume_profile = df.groupby('price_bin')['volume'].sum()
        df['market_profile_strength'] = df['price_bin'].map(volume_profile)
        return df


    # =============================================================
    # 🧭 SUPPORT / RESISTANCE LEVELS (local lows/highs)
    # =============================================================
    def _add_support_resistance(self, df, window=10):
        df['support'] = df['low'].rolling(window=window).min()
        df['resistance'] = df['high'].rolling(window=window).max()
        df['price_position'] = (df['close'] - df['support']) / (df['resistance'] - df['support'] + 1e-10)
        return df


In [16]:
class AdvancedFeatureEngineer:
    def add_all_features(self, df):
        print("add_all_features is defined!")
        return df

# Test
engineer = AdvancedFeatureEngineer()
engineer.add_all_features(pd.DataFrame())  # Should print the message


add_all_features is defined!


""


In [17]:
engineer = AdvancedFeatureEngineer()
df_features = engineer.add_all_features(df_stock)  # Use your actual DataFrame variable here
print(df_features.head())


add_all_features is defined!
                                close        high         low        open  \
Datetime                                                                    
2025-10-01 03:45:00+00:00  872.849976  874.700012  871.200012  874.700012   
2025-10-01 03:50:00+00:00  874.849976  876.349976  872.099976  872.849976   
2025-10-01 03:55:00+00:00  871.000000  875.450012  870.400024  874.849976   
2025-10-01 04:00:00+00:00  871.150024  871.849976  870.650024  871.099976   
2025-10-01 04:05:00+00:00  873.000000  873.299988  870.849976  871.099976   

                           volume  
Datetime                           
2025-10-01 03:45:00+00:00       0  
2025-10-01 03:50:00+00:00  154136  
2025-10-01 03:55:00+00:00  131612  
2025-10-01 04:00:00+00:00   69056  
2025-10-01 04:05:00+00:00   85664  


In [19]:
# Assuming AdvancedFeatureEngineer class is already defined and imported

engineer = AdvancedFeatureEngineer()

# Add features
df_features = engineer.add_all_features(df_stock)

# Show sample of enriched data
print(df_features.head())


add_all_features is defined!
                 close        high         low        open    volume
Date                                                                
2024-10-07  755.480286  788.173789  750.333628  784.252526  24267990
2024-10-08  766.067688  768.959631  757.048772  757.048772   8560960
2024-10-09  781.703735  788.516881  766.655854  771.214346  18758697
2024-10-10  781.409607  788.614963  779.350968  783.272231  10241015
2024-10-11  784.007446  786.899389  777.390316  781.507653  14322720


In [18]:
import yfinance as yf

# Fetch 1 year of daily data for SBI (SBIN.NS)
df_stock = yf.download('SBIN.NS', period='1y', interval='1d', progress=False)

# Check if data was fetched
if df_stock.empty:
    print("Failed to fetch SBI data.")
else:
    # If columns are MultiIndex, flatten them by taking first level
    if isinstance(df_stock.columns, pd.MultiIndex):
        df_stock.columns = df_stock.columns.get_level_values(0)

    # Convert column names to lowercase for consistency
    df_stock.columns = [col.lower() for col in df_stock.columns]

    # Show first few rows to verify
    print(df_stock.head())


                 close        high         low        open    volume
Date                                                                
2024-10-07  755.480286  788.173789  750.333628  784.252526  24267990
2024-10-08  766.067688  768.959631  757.048772  757.048772   8560960
2024-10-09  781.703735  788.516881  766.655854  771.214346  18758697
2024-10-10  781.409607  788.614963  779.350968  783.272231  10241015
2024-10-11  784.007446  786.899389  777.390316  781.507653  14322720


In [20]:
ticker = yf.Ticker('SBIN.NS')
data = ticker.history(period='1mo')  # Fetches 1 month of historical data
print(data)


                                 Open        High         Low       Close  \
Date                                                                        
2025-09-08 00:00:00+05:30  808.000000  813.849976  806.799988  808.799988   
2025-09-09 00:00:00+05:30  812.000000  812.349976  805.599976  808.849976   
2025-09-10 00:00:00+05:30  812.000000  824.599976  810.400024  818.200012   
2025-09-11 00:00:00+05:30  819.099976  825.700012  819.000000  823.650024   
2025-09-12 00:00:00+05:30  824.099976  825.799988  819.799988  823.549988   
2025-09-15 00:00:00+05:30  823.549988  827.799988  821.099976  824.750000   
2025-09-16 00:00:00+05:30  825.099976  833.000000  821.650024  831.549988   
2025-09-17 00:00:00+05:30  834.299988  858.150024  831.000000  857.150024   
2025-09-18 00:00:00+05:30  858.650024  860.799988  851.099976  854.349976   
2025-09-19 00:00:00+05:30  852.000000  864.450012  849.299988  862.349976   
2025-09-22 00:00:00+05:30  862.000000  868.200012  854.000000  855.250000   

In [21]:
data2 = yf.download('SBIN.NS', period='1mo')  # Also fetches 1 month historical data
print(data2)


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker         SBIN.NS     SBIN.NS     SBIN.NS     SBIN.NS   SBIN.NS
Date                                                                
2025-09-08  808.799988  813.849976  806.799988  808.000000   4614227
2025-09-09  808.849976  812.349976  805.599976  812.000000   4567272
2025-09-10  818.200012  824.599976  810.400024  812.000000   7285853
2025-09-11  823.650024  825.700012  819.000000  819.099976   7425055
2025-09-12  823.549988  825.799988  819.799988  824.099976   5078018
2025-09-15  824.750000  827.799988  821.099976  823.549988   3852494
2025-09-16  831.549988  833.000000  821.650024  825.099976   9467531
2025-09-17  857.150024  858.150024  831.000000  834.299988  17243938
2025-09-18  854.349976  860.799988  851.099976  858.650024  10400847
2025-09-19  862.349976  864.450012  849.299988  852.000000  15206426
2025-09-22  855.250000  868.200012  854.000000  862.000000   6594034
2025-09-23  870.599976  874.250000

In [22]:
ticker = yf.Ticker('SBIN.NS')
data = ticker.history(period='1mo')  # Fetches 1 month of historical data
print(data)


                                 Open        High         Low       Close  \
Date                                                                        
2025-09-08 00:00:00+05:30  808.000000  813.849976  806.799988  808.799988   
2025-09-09 00:00:00+05:30  812.000000  812.349976  805.599976  808.849976   
2025-09-10 00:00:00+05:30  812.000000  824.599976  810.400024  818.200012   
2025-09-11 00:00:00+05:30  819.099976  825.700012  819.000000  823.650024   
2025-09-12 00:00:00+05:30  824.099976  825.799988  819.799988  823.549988   
2025-09-15 00:00:00+05:30  823.549988  827.799988  821.099976  824.750000   
2025-09-16 00:00:00+05:30  825.099976  833.000000  821.650024  831.549988   
2025-09-17 00:00:00+05:30  834.299988  858.150024  831.000000  857.150024   
2025-09-18 00:00:00+05:30  858.650024  860.799988  851.099976  854.349976   
2025-09-19 00:00:00+05:30  852.000000  864.450012  849.299988  862.349976   
2025-09-22 00:00:00+05:30  862.000000  868.200012  854.000000  855.250000   

In [ ]:
import ta
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def add_technical_indicators(df):
    """
    Add comprehensive technical indicators to stock dataframe

    Parameters:
    df (pd.DataFrame): DataFrame with OHLCV data (Open, High, Low, Close, Volume)

    Returns:
    pd.DataFrame: DataFrame with added technical indicators
    """

    # ========== TREND INDICATORS ==========

    # Simple Moving Average (SMA)
    df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['SMA_50'] = ta.trend.sma_indicator(df['Close'], window=50)
    df['SMA_100'] = ta.trend.sma_indicator(df['Close'], window=100)
    df['SMA_200'] = ta.trend.sma_indicator(df['Close'], window=200)

    # Exponential Moving Average (EMA)
    df['EMA_12'] = ta.trend.ema_indicator(df['Close'], window=12)
    df['EMA_20'] = ta.trend.ema_indicator(df['Close'], window=20)
    df['EMA_26'] = ta.trend.ema_indicator(df['Close'], window=26)
    df['EMA_50'] = ta.trend.ema_indicator(df['Close'], window=50)

    # MACD (Moving Average Convergence Divergence)
    df['MACD'] = ta.trend.macd(df['Close'])
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])
    df['MACD_Hist'] = ta.trend.macd_diff(df['Close'])

    # ADX (Average Directional Index)
    df['ADX'] = ta.trend.adx(df['High'], df['Low'], df['Close'], window=14)
    df['ADX_Pos'] = ta.trend.adx_pos(df['High'], df['Low'], df['Close'], window=14)
    df['ADX_Neg'] = ta.trend.adx_neg(df['High'], df['Low'], df['Close'], window=14)

    # Trend Lines (Linear Regression)
    df['Trend_Line'] = ta.trend.sma_indicator(df['Close'], window=20)


    # ========== MOMENTUM INDICATORS ==========

    # RSI (Relative Strength Index)
    df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)
    df['RSI_7'] = ta.momentum.rsi(df['Close'], window=7)

    # Stochastic Oscillator
    df['Stoch_K'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], window=14)
    df['Stoch_D'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], window=14)

    # DeMarker Indicator (custom implementation)
    def demarker(high, low, period=14):
        demax = high.diff()
        demax[demax < 0] = 0
        demin = -low.diff()
        demin[demin < 0] = 0
        demax_ma = demax.rolling(window=period).mean()
        demin_ma = demin.rolling(window=period).mean()
        return demax_ma / (demax_ma + demin_ma)

    df['DeMarker'] = demarker(df['High'], df['Low'], period=14)


    # ========== VOLATILITY INDICATORS ==========

    # ATR (Average True Range)
    df['ATR_14'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=14)
    df['ATR_7'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=7)

    # Bollinger Bands
    bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Mid'] = bb.bollinger_mavg()
    df['BB_Low'] = bb.bollinger_lband()
    df['BB_Width'] = bb.bollinger_wband()
    df['BB_Pct'] = bb.bollinger_pband()

    # Standard Deviation
    df['Std_Dev_20'] = df['Close'].rolling(window=20).std()
    df['Std_Dev_50'] = df['Close'].rolling(window=50).std()


    # ========== VOLUME INDICATORS ==========

    # OBV (On-Balance Volume)
    df['OBV'] = ta.volume.on_balance_volume(df['Close'], df['Volume'])

    # Accumulation/Distribution Index
    df['ADI'] = ta.volume.acc_dist_index(df['High'], df['Low'], df['Close'], df['Volume'])

    # Accumulation/Distribution Line (alternative calculation)
    df['AD_Line'] = ta.volume.acc_dist_index(df['High'], df['Low'], df['Close'], df['Volume'])


    # ========== SUPPORT/RESISTANCE & PIVOT POINTS ==========

    # Pivot Points (Standard)
    df['Pivot'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['R1'] = 2 * df['Pivot'] - df['Low']
    df['S1'] = 2 * df['Pivot'] - df['High']
    df['R2'] = df['Pivot'] + (df['High'] - df['Low'])
    df['S2'] = df['Pivot'] - (df['High'] - df['Low'])
    df['R3'] = df['High'] + 2 * (df['Pivot'] - df['Low'])
    df['S3'] = df['Low'] - 2 * (df['High'] - df['Pivot'])

    # Support and Resistance Levels (using rolling max/min)
    df['Resistance_20'] = df['High'].rolling(window=20).max()
    df['Support_20'] = df['Low'].rolling(window=20).min()
    df['Resistance_50'] = df['High'].rolling(window=50).max()
    df['Support_50'] = df['Low'].rolling(window=50).min()


    # ========== MARKET PROFILE (Volume Profile Approximation) ==========

    # VWAP (Volume Weighted Average Price)
    df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()

    # Volume-weighted moving average
    df['VWMA_20'] = (df['Close'] * df['Volume']).rolling(window=20).sum() / df['Volume'].rolling(window=20).sum()


    # Drop NaN values created by rolling indicators
    df.dropna(inplace=True)

    return df


def analyze_SBI_stock(period='1y'):
    """
    Fetch SBIN.NS stock data and apply technical indicators

    Parameters:
    period (str): Time period for data ('1mo', '3mo', '6mo', '1y', '2y', '5y', 'max')

    Returns:
    pd.DataFrame: DataFrame with stock data and technical indicators
    """
    print("Fetching SBIN.NS stock data...")

    try:
        # Download SBI stock data
        ticker = yf.Ticker('SBIN.NS')
        SBI = ticker.history(period=period)

        # Reset column names if multi-level
        if isinstance(SBI.columns, pd.MultiIndex):
            SBI.columns = SBI.columns.get_level_values(0)

        if SBI.empty:
            print("Error: Could not fetch SBIN.NS data")
            print("Trying alternative methods...")

            # Try with different date range
            end_date = datetime.now()
            start_date = end_date - timedelta(days=365)
            SBI = yf.download('SBIN.NS', start=start_date, end=end_date, progress=False)

            if SBI.empty:
                print("Failed to fetch data. Please check:")
                print("1. Internet connection")
                print("2. Symbol 'SBIN.NS' is correct")
                print("3. Try running: pip install --upgrade yfinance")
                return None
    except Exception as e:
        print(f"Error fetching data: {str(e)}")
        return None

    print(f"Data fetched: {len(SBI)} rows from {SBI.index[0].strftime('%Y-%m-%d')} to {SBI.index[-1].strftime('%Y-%m-%d')}")
    print(f"Columns: {list(SBI.columns)}")

    # Ensure we have the required columns
    required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    missing_cols = [col for col in required_cols if col not in SBI.columns]
    if missing_cols:
        print(f"Error: Missing required columns: {missing_cols}")
        return None

    # Add technical indicators
    print("Adding technical indicators...")
    SBI_with_indicators = add_technical_indicators(SBI)

    print(f"\nTotal columns: {len(SBI_with_indicators.columns)}")
    print(f"Total rows after cleaning: {len(SBI_with_indicators)}")

    # Display latest data
    print("\n" + "="*80)
    print("LATEST SBIN.NS STOCK DATA WITH TECHNICAL INDICATORS")
    print("="*80)

    latest = SBI_with_indicators.iloc[-1]

    print(f"\nDate: {SBI_with_indicators.index[-1].strftime('%Y-%m-%d')}")
    print(f"\n--- PRICE DATA ---")
    print(f"Close Price: ₹{latest['Close']:.2f}")
    print(f"Open: ₹{latest['Open']:.2f} | High: ₹{latest['High']:.2f} | Low: ₹{latest['Low']:.2f}")
    print(f"Volume: {latest['Volume']:,.0f}")

    print(f"\n--- TREND INDICATORS ---")
    print(f"SMA 20: ₹{latest['SMA_20']:.2f} | SMA 50: ₹{latest['SMA_50']:.2f}")
    print(f"EMA 20: ₹{latest['EMA_20']:.2f} | EMA 50: ₹{latest['EMA_50']:.2f}")
    print(f"MACD: {latest['MACD']:.2f} | Signal: {latest['MACD_Signal']:.2f} | Hist: {latest['MACD_Hist']:.2f}")
    print(f"ADX: {latest['ADX']:.2f}")

    print(f"\n--- MOMENTUM INDICATORS ---")
    print(f"RSI (14): {latest['RSI_14']:.2f}")
    print(f"Stochastic K: {latest['Stoch_K']:.2f} | D: {latest['Stoch_D']:.2f}")
    print(f"DeMarker: {latest['DeMarker']:.2f}")

    print(f"\n--- VOLATILITY INDICATORS ---")
    print(f"ATR (14): ₹{latest['ATR_14']:.2f}")
    print(f"Bollinger Upper: ₹{latest['BB_High']:.2f} | Middle: ₹{latest['BB_Mid']:.2f} | Lower: ₹{latest['BB_Low']:.2f}")
    print(f"Std Dev (20): ₹{latest['Std_Dev_20']:.2f}")

    print(f"\n--- VOLUME INDICATORS ---")
    print(f"OBV: {latest['OBV']:,.0f}")
    print(f"ADI: {latest['ADI']:,.2f}")

    print(f"\n--- SUPPORT & RESISTANCE ---")
    print(f"Pivot: ₹{latest['Pivot']:.2f}")
    print(f"Resistance: R1=₹{latest['R1']:.2f} | R2=₹{latest['R2']:.2f} | R3=₹{latest['R3']:.2f}")
    print(f"Support: S1=₹{latest['S1']:.2f} | S2=₹{latest['S2']:.2f} | S3=₹{latest['S3']:.2f}")
    print(f"VWAP: ₹{latest['VWAP']:.2f}")

    print("\n" + "="*80)

    return SBI_with_indicators


def plot_technical_analysis(df, ticker_name='SBIN.NS'):
    """
    Create interactive Plotly charts for technical analysis

    Parameters:
    df (pd.DataFrame): DataFrame with technical indicators
    ticker_name (str): Stock ticker name for title
    """

    # Create subplots
    fig = make_subplots(
        rows=6, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=(
            f'{ticker_name} - Price & Moving Averages',
            'MACD',
            'RSI & Stochastic',
            'Bollinger Bands',
            'Volume & OBV',
            'ADX & ATR'
        ),
        row_heights=[0.3, 0.15, 0.15, 0.15, 0.15, 0.1]
    )

    # === ROW 1: Candlestick Chart with Moving Averages ===
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='OHLC',
            showlegend=False
        ),
        row=1, col=1
    )

    # Add Moving Averages
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], name='SMA 20', line=dict(color='orange', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='SMA 50', line=dict(color='blue', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_20'], name='EMA 20', line=dict(color='purple', width=1, dash='dash')), row=1, col=1)

    # Add Bollinger Bands
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_High'], name='BB Upper', line=dict(color='gray', width=1, dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Low'], name='BB Lower', line=dict(color='gray', width=1, dash='dot'), fill='tonexty', fillcolor='rgba(128,128,128,0.1)'), row=1, col=1)

    # Add Support & Resistance
    fig.add_trace(go.Scatter(x=df.index, y=df['Resistance_20'], name='Resistance', line=dict(color='red', width=1, dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Support_20'], name='Support', line=dict(color='green', width=1, dash='dot')), row=1, col=1)

    # === ROW 2: MACD ===
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue', width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_Signal'], name='Signal', line=dict(color='orange', width=1)), row=2, col=1)

    # MACD Histogram
    colors = ['red' if val < 0 else 'green' for val in df['MACD_Hist']]
    fig.add_trace(go.Bar(x=df.index, y=df['MACD_Hist'], name='MACD Hist', marker_color=colors, showlegend=False), row=2, col=1)

    # === ROW 3: RSI & Stochastic ===
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI_14'], name='RSI 14', line=dict(color='purple', width=2)), row=3, col=1)

    # Add RSI levels
    fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=3, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", opacity=0.3, row=3, col=1)

    # Stochastic
    fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_K'], name='Stoch K', line=dict(color='blue', width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_D'], name='Stoch D', line=dict(color='orange', width=1)), row=3, col=1)

    # === ROW 4: Bollinger Bands Width & Percentage ===
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Width'], name='BB Width', line=dict(color='brown', width=1)), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Pct'], name='BB %', line=dict(color='teal', width=1)), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Std_Dev_20'], name='Std Dev', line=dict(color='magenta', width=1)), row=4, col=1)

    # === ROW 5: Volume & OBV ===
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume', marker_color='lightblue', showlegend=False), row=5, col=1)

    # OBV on secondary y-axis (normalized)
    obv_normalized = (df['OBV'] - df['OBV'].min()) / (df['OBV'].max() - df['OBV'].min()) * df['Volume'].max()
    fig.add_trace(go.Scatter(x=df.index, y=obv_normalized, name='OBV (normalized)', line=dict(color='darkblue', width=2)), row=5, col=1)

    # === ROW 6: ADX & ATR ===
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX'], name='ADX', line=dict(color='red', width=2)), row=6, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX_Pos'], name='ADX+', line=dict(color='green', width=1)), row=6, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX_Neg'], name='ADX-', line=dict(color='red', width=1)), row=6, col=1)

    # Add ADX threshold
    fig.add_hline(y=25, line_dash="dash", line_color="gray", opacity=0.5, row=6, col=1)

    # Update layout
    fig.update_layout(
        title=f'{ticker_name} - Comprehensive Technical Analysis',
        height=1800,
        showlegend=True,
        xaxis_rangeslider_visible=False,
        hovermode='x unified',
        template='plotly_white'
    )

    # Update y-axes labels
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="RSI/Stoch", row=3, col=1)
    fig.update_yaxes(title_text="BB Width", row=4, col=1)
    fig.update_yaxes(title_text="Volume", row=5, col=1)
    fig.update_yaxes(title_text="ADX", row=6, col=1)

    fig.update_xaxes(title_text="Date", row=6, col=1)

    return fig


def plot_pivot_points(df, ticker_name='SBIN.NS', days=30):
    """
    Create Plotly chart for Pivot Points and Support/Resistance levels

    Parameters:
    df (pd.DataFrame): DataFrame with technical indicators
    ticker_name (str): Stock ticker name
    days (int): Number of recent days to display
    """

    # Get recent data
    recent_df = df.tail(days)

    fig = go.Figure()

    # Candlestick
    fig.add_trace(go.Candlestick(
        x=recent_df.index,
        open=recent_df['Open'],
        high=recent_df['High'],
        low=recent_df['Low'],
        close=recent_df['Close'],
        name='OHLC'
    ))

    # Pivot Points
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['Pivot'], name='Pivot', line=dict(color='yellow', width=2, dash='dash')))

    # Resistance Levels
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['R1'], name='R1', line=dict(color='red', width=1)))
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['R2'], name='R2', line=dict(color='darkred', width=1)))
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['R3'], name='R3', line=dict(color='firebrick', width=1)))

    # Support Levels
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['S1'], name='S1', line=dict(color='green', width=1)))
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['S2'], name='S2', line=dict(color='darkgreen', width=1)))
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['S3'], name='S3', line=dict(color='forestgreen', width=1)))

    # VWAP
    fig.add_trace(go.Scatter(x=recent_df.index, y=recent_df['VWAP'], name='VWAP', line=dict(color='purple', width=2)))

    fig.update_layout(
        title=f'{ticker_name} - Pivot Points & Support/Resistance (Last {days} Days)',
        xaxis_title='Date',
        yaxis_title='Price (₹)',
        height=600,
        hovermode='x unified',
        template='plotly_white',
        xaxis_rangeslider_visible=False
    )

    return fig


def plot_volume_profile(df, ticker_name='SBIN.NS'):
    """
    Create Volume Profile (Market Profile approximation)

    Parameters:
    df (pd.DataFrame): DataFrame with technical indicators
    ticker_name (str): Stock ticker name
    """

    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.7, 0.3],
        subplot_titles=(f'{ticker_name} Price Action', 'Volume Profile'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Left: Price chart
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='OHLC'
        ),
        row=1, col=1
    )

    fig.add_trace(go.Scatter(x=df.index, y=df['VWAP'], name='VWAP', line=dict(color='orange', width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['VWMA_20'], name='VWMA 20', line=dict(color='blue', width=2)), row=1, col=1)

    # Right: Volume Profile (horizontal volume distribution)
    price_bins = np.linspace(df['Low'].min(), df['High'].max(), 50)
    volume_at_price = []

    for i in range(len(price_bins) - 1):
        mask = (df['Close'] >= price_bins[i]) & (df['Close'] < price_bins[i + 1])
        volume_at_price.append(df.loc[mask, 'Volume'].sum())

    fig.add_trace(
        go.Bar(
            x=volume_at_price,
            y=price_bins[:-1],
            orientation='h',
            name='Volume Profile',
            marker_color='lightblue'
        ),
        row=1, col=2
    )

    fig.update_layout(
        title=f'{ticker_name} - Volume Profile Analysis',
        height=600,
        showlegend=True,
        template='plotly_white',
        xaxis_rangeslider_visible=False
    )

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Volume", row=1, col=2)
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Price (₹)", row=1, col=2)

    return fig


def plot_momentum_indicators(df, ticker_name='SBIN.NS'):
    """
    Create detailed momentum indicators chart
    """
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Price Action',
            'RSI (Relative Strength Index)',
            'Stochastic Oscillator',
            'DeMarker Indicator'
        ),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    # Row 1: Price
    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                                  low=df['Low'], close=df['Close'], name='Price'), row=1, col=1)

    # Row 2: RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI_14'], name='RSI 14',
                            line=dict(color='purple', width=2)), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI_7'], name='RSI 7',
                            line=dict(color='blue', width=1)), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
    fig.add_hrect(y0=70, y1=100, fillcolor="red", opacity=0.1, row=2, col=1)
    fig.add_hrect(y0=0, y1=30, fillcolor="green", opacity=0.1, row=2, col=1)

    # Row 3: Stochastic
    fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_K'], name='%K',
                            line=dict(color='blue', width=2)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Stoch_D'], name='%D',
                            line=dict(color='orange', width=2)), row=3, col=1)
    fig.add_hline(y=80, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=20, line_dash="dash", line_color="green", row=3, col=1)

    # Row 4: DeMarker
    fig.add_trace(go.Scatter(x=df.index, y=df['DeMarker'], name='DeMarker',
                            line=dict(color='teal', width=2)), row=4, col=1)
    fig.add_hline(y=0.7, line_dash="dash", line_color="red", row=4, col=1)
    fig.add_hline(y=0.3, line_dash="dash", line_color="green", row=4, col=1)

    fig.update_layout(title=f'{ticker_name} - Momentum Indicators', height=1000,
                     template='plotly_white', showlegend=True, hovermode='x unified')
    fig.update_xaxes(title_text="Date", row=4, col=1)
    return fig


def plot_volatility_analysis(df, ticker_name='SBIN.NS'):
    """
    Create volatility analysis charts
    """
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Bollinger Bands with Price',
            'ATR (Average True Range)',
            'Standard Deviation & BB Width'
        ),
        row_heights=[0.5, 0.25, 0.25]
    )

    # Row 1: Bollinger Bands
    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                                  low=df['Low'], close=df['Close'], name='Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_High'], name='BB Upper',
                            line=dict(color='red', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Mid'], name='BB Middle',
                            line=dict(color='blue', width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Low'], name='BB Lower',
                            line=dict(color='green', width=1), fill='tonexty'), row=1, col=1)

    # Row 2: ATR
    fig.add_trace(go.Scatter(x=df.index, y=df['ATR_14'], name='ATR 14',
                            line=dict(color='orange', width=2), fill='tozeroy'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ATR_7'], name='ATR 7',
                            line=dict(color='red', width=1)), row=2, col=1)

    # Row 3: Std Dev & BB Width
    fig.add_trace(go.Scatter(x=df.index, y=df['Std_Dev_20'], name='Std Dev 20',
                            line=dict(color='purple', width=2)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Width'], name='BB Width',
                            line=dict(color='brown', width=2)), row=3, col=1)

    fig.update_layout(title=f'{ticker_name} - Volatility Analysis', height=900,
                     template='plotly_white', showlegend=True, hovermode='x unified')
    fig.update_xaxes(title_text="Date", row=3, col=1)
    return fig


def plot_trend_analysis(df, ticker_name='SBIN.NS'):
    """
    Create trend analysis with moving averages and ADX
    """
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Price with Moving Averages',
            'MACD Analysis',
            'ADX - Trend Strength'
        ),
        row_heights=[0.5, 0.25, 0.25]
    )

    # Row 1: Price with MAs
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close',
                            line=dict(color='black', width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], name='SMA 20',
                            line=dict(color='blue', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='SMA 50',
                            line=dict(color='orange', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_100'], name='SMA 100',
                            line=dict(color='red', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], name='SMA 200',
                            line=dict(color='purple', width=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA_20'], name='EMA 20',
                            line=dict(color='cyan', width=1, dash='dash')), row=1, col=1)

    # Row 2: MACD
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD',
                            line=dict(color='blue', width=2)), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_Signal'], name='Signal',
                            line=dict(color='red', width=2)), row=2, col=1)
    colors = ['red' if val < 0 else 'green' for val in df['MACD_Hist']]
    fig.add_trace(go.Bar(x=df.index, y=df['MACD_Hist'], name='Histogram',
                        marker_color=colors), row=2, col=1)

    # Row 3: ADX
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX'], name='ADX',
                            line=dict(color='black', width=3)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX_Pos'], name='+DI',
                            line=dict(color='green', width=2)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['ADX_Neg'], name='-DI',
                            line=dict(color='red', width=2)), row=3, col=1)
    fig.add_hline(y=25, line_dash="dash", line_color="blue", row=3, col=1)
    fig.add_hrect(y0=25, y1=50, fillcolor="yellow", opacity=0.1, row=3, col=1)
    fig.add_hrect(y0=50, y1=100, fillcolor="green", opacity=0.1, row=3, col=1)

    fig.update_layout(title=f'{ticker_name} - Trend Analysis', height=1000,
                     template='plotly_white', showlegend=True, hovermode='x unified')
    fig.update_xaxes(title_text="Date", row=3, col=1)
    return fig


def plot_volume_analysis(df, ticker_name='SBIN.NS'):
    """
    Create volume analysis charts
    """
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Price Action',
            'Volume',
            'OBV (On-Balance Volume)',
            'Accumulation/Distribution'
        ),
        row_heights=[0.35, 0.2, 0.25, 0.2]
    )

    # Row 1: Price
    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                                  low=df['Low'], close=df['Close'], name='Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['VWAP'], name='VWAP',
                            line=dict(color='orange', width=2)), row=1, col=1)

    # Row 2: Volume
    colors = ['red' if df['Close'].iloc[i] < df['Open'].iloc[i] else 'green'
              for i in range(len(df))]
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume',
                        marker_color=colors), row=2, col=1)

    # Row 3: OBV
    fig.add_trace(go.Scatter(x=df.index, y=df['OBV'], name='OBV',
                            line=dict(color='blue', width=2), fill='tozeroy'), row=3, col=1)

    # Row 4: A/D
    fig.add_trace(go.Scatter(x=df.index, y=df['ADI'], name='A/D Index',
                            line=dict(color='purple', width=2)), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['AD_Line'], name='A/D Line',
                            line=dict(color='teal', width=1, dash='dash')), row=4, col=1)

    fig.update_layout(title=f'{ticker_name} - Volume Analysis', height=1000,
                     template='plotly_white', showlegend=True, hovermode='x unified')
    fig.update_xaxes(title_text="Date", row=4, col=1)
    return fig


def plot_support_resistance_heatmap(df, ticker_name='SBIN.NS'):
    """
    Create support and resistance heatmap
    """
    fig = go.Figure()

    # Candlestick
    fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],
                                  low=df['Low'], close=df['Close'], name='Price'))

    # Dynamic S/R levels
    fig.add_trace(go.Scatter(x=df.index, y=df['Resistance_20'], name='R 20D',
                            line=dict(color='red', width=2)))
    fig.add_trace(go.Scatter(x=df.index, y=df['Resistance_50'], name='R 50D',
                            line=dict(color='darkred', width=2, dash='dash')))
    fig.add_trace(go.Scatter(x=df.index, y=df['Support_20'], name='S 20D',
                            line=dict(color='green', width=2)))
    fig.add_trace(go.Scatter(x=df.index, y=df['Support_50'], name='S 50D',
                            line=dict(color='darkgreen', width=2, dash='dash')))

    # Pivot levels
    fig.add_trace(go.Scatter(x=df.index, y=df['Pivot'], name='Pivot',
                            line=dict(color='yellow', width=2)))
    fig.add_trace(go.Scatter(x=df.index, y=df['R1'], name='R1',
                            line=dict(color='orange', width=1, dash='dot')))
    fig.add_trace(go.Scatter(x=df.index, y=df['S1'], name='S1',
                            line=dict(color='lightgreen', width=1, dash='dot')))

    fig.update_layout(
        title=f'{ticker_name} - Support & Resistance Levels',
        xaxis_title='Date',
        yaxis_title='Price (₹)',
        height=700,
        template='plotly_white',
        hovermode='x unified',
        xaxis_rangeslider_visible=False
    )

    return fig


def plot_correlation_matrix(df, ticker_name='SBIN.NS'):
    """
    Create correlation matrix heatmap for indicators
    """
    # Select key indicators
    indicators = ['Close', 'SMA_20', 'SMA_50', 'RSI_14', 'MACD', 'ADX',
                  'BB_Width', 'ATR_14', 'Volume', 'OBV', 'Stoch_K']

    corr_data = df[indicators].corr()

    fig = go.Figure(data=go.Heatmap(
        z=corr_data.values,
        x=corr_data.columns,
        y=corr_data.columns,
        colorscale='RdBu',
        zmid=0,
        text=corr_data.values.round(2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title="Correlation")
    ))

    fig.update_layout(
        title=f'{ticker_name} - Technical Indicators Correlation Matrix',
        height=700,
        width=800,
        template='plotly_white'
    )

    return fig


def plot_price_distribution(df, ticker_name='SBIN.NS'):
    """
    Create price distribution and returns analysis
    """
    # Calculate returns
    df['Returns'] = df['Close'].pct_change() * 100

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Price Distribution',
            'Daily Returns Distribution',
            'Cumulative Returns',
            'Volume Distribution'
        ),
        specs=[[{"type": "histogram"}, {"type": "histogram"}],
               [{"type": "scatter"}, {"type": "histogram"}]]
    )

    # Price Distribution
    fig.add_trace(go.Histogram(x=df['Close'], name='Price', nbinsx=50,
                              marker_color='blue'), row=1, col=1)

    # Returns Distribution
    fig.add_trace(go.Histogram(x=df['Returns'].dropna(), name='Returns', nbinsx=50,
                              marker_color='green'), row=1, col=2)

    # Cumulative Returns
    cum_returns = (1 + df['Returns']/100).cumprod()
    fig.add_trace(go.Scatter(x=df.index, y=cum_returns, name='Cumulative Returns',
                            line=dict(color='purple', width=2), fill='tozeroy'), row=2, col=1)

    # Volume Distribution
    fig.add_trace(go.Histogram(x=df['Volume'], name='Volume', nbinsx=50,
                              marker_color='orange'), row=2, col=2)

    fig.update_layout(
        title=f'{ticker_name} - Statistical Analysis',
        height=800,
        template='plotly_white',
        showlegend=False
    )

    return fig


# Run the analysis
if __name__ == "__main__":
    SBI_data = analyze_SBI_stock(period='1y')

    if SBI_data is not None:
        # Optional: Save to CSV
        SBI_data.to_csv('SBI_technical_indicators.csv')
        print("\nData saved to 'SBI_technical_indicators.csv'")

        # Display first few rows
        print("\nFirst 5 rows of data:")
        print(SBI_data.head())

        print("\nAll available columns:")
        for i, col in enumerate(SBI_data.columns, 1):
            print(f"{i}. {col}")

        print("\n" + "="*80)
        print("GENERATING INTERACTIVE PLOTLY CHARTS...")
        print("="*80)

        # Generate and save all Plotly charts
        charts = []

        print("\n1. Creating comprehensive technical analysis chart...")
        fig1 = plot_technical_analysis(SBI_data, 'SBIN.NS')
        fig1.write_html('SBI_01_technical_analysis.html')
        charts.append(('SBI_01_technical_analysis.html', 'Complete Technical Analysis'))
        print("   ✓ Saved: SBI_01_technical_analysis.html")

        print("\n2. Creating pivot points chart...")
        fig2 = plot_pivot_points(SBI_data, 'SBIN.NS', days=60)
        fig2.write_html('SBI_02_pivot_points.html')
        charts.append(('SBI_02_pivot_points.html', 'Pivot Points & S/R Levels'))
        print("   ✓ Saved: SBI_02_pivot_points.html")

        print("\n3. Creating volume profile chart...")
        fig3 = plot_volume_profile(SBI_data, 'SBIN.NS')
        fig3.write_html('SBI_03_volume_profile.html')
        charts.append(('SBI_03_volume_profile.html', 'Volume Profile Analysis'))
        print("   ✓ Saved: SBI_03_volume_profile.html")

        print("\n4. Creating momentum indicators chart...")
        fig4 = plot_momentum_indicators(SBI_data, 'SBIN.NS')
        fig4.write_html('SBI_04_momentum_indicators.html')
        charts.append(('SBI_04_momentum_indicators.html', 'Momentum Indicators (RSI, Stochastic, DeMarker)'))
        print("   ✓ Saved: SBI_04_momentum_indicators.html")

        print("\n5. Creating volatility analysis chart...")
        fig5 = plot_volatility_analysis(SBI_data, 'SBIN.NS')
        fig5.write_html('SBI_05_volatility_analysis.html')
        charts.append(('SBI_05_volatility_analysis.html', 'Volatility Analysis (BB, ATR, Std Dev)'))
        print("   ✓ Saved: SBI_05_volatility_analysis.html")

        print("\n6. Creating trend analysis chart...")
        fig6 = plot_trend_analysis(SBI_data, 'SBIN.NS')
        fig6.write_html('SBI_06_trend_analysis.html')
        charts.append(('SBI_06_trend_analysis.html', 'Trend Analysis (MAs, MACD, ADX)'))
        print("   ✓ Saved: SBI_06_trend_analysis.html")

        print("\n7. Creating volume analysis chart...")
        fig7 = plot_volume_analysis(SBI_data, 'SBIN.NS')
        fig7.write_html('SBI_07_volume_analysis.html')
        charts.append(('SBI_07_volume_analysis.html', 'Volume Analysis (OBV, A/D)'))
        print("   ✓ Saved: SBI_07_volume_analysis.html")

        print("\n8. Creating support & resistance heatmap...")
        fig8 = plot_support_resistance_heatmap(SBI_data, 'SBIN.NS')
        fig8.write_html('SBI_08_support_resistance.html')
        charts.append(('SBI_08_support_resistance.html', 'Support & Resistance Heatmap'))
        print("   ✓ Saved: SBI_08_support_resistance.html")

        print("\n9. Creating correlation matrix...")
        fig9 = plot_correlation_matrix(SBI_data, 'SBIN.NS')
        fig9.write_html('SBI_09_correlation_matrix.html')
        charts.append(('SBI_09_correlation_matrix.html', 'Indicators Correlation Matrix'))
        print("   ✓ Saved: SBI_09_correlation_matrix.html")

        print("\n10. Creating statistical analysis chart...")
        fig10 = plot_price_distribution(SBI_data, 'SBIN.NS')
        fig10.write_html('SBI_10_statistical_analysis.html')
        charts.append(('SBI_10_statistical_analysis.html', 'Statistical Analysis & Distribution'))
        print("   ✓ Saved: SBI_10_statistical_analysis.html")

        print("\n" + "="*80)
        print("✓ ALL CHARTS GENERATED SUCCESSFULLY!")
        print("="*80)

        print("\n📊 INTERACTIVE HTML FILES CREATED:")
        print("-" * 80)
        for i, (filename, description) in enumerate(charts, 1):
            print(f"  {i:2d}. {filename:40s} - {description}")

        print("\n" + "="*80)
        print("📈 QUICK SUMMARY")
        print("="*80)
        print(f"Total Data Points: {len(SBI_data)}")
        print(f"Date Range: {SBI_data.index[0].strftime('%Y-%m-%d')} to {SBI_data.index[-1].strftime('%Y-%m-%d')}")
        print(f"Total Indicators: {len(SBI_data.columns)}")
        print(f"Charts Generated: {len(charts)}")
        print(f"\nLatest Close Price: ₹{SBI_data['Close'].iloc[-1]:.2f}")
        print(f"52-Week High: ₹{SBI_data['High'].tail(252).max():.2f}")
        print(f"52-Week Low: ₹{SBI_data['Low'].tail(252).min():.2f}")

        # Show all charts
        print("\n" + "="*80)
        print("Opening charts in browser...")
        print("="*80)

        fig1.show()
        fig2.show()
        fig3.show()
        fig4.show()
        fig5.show()
        fig6.show()
        fig7.show()
        fig8.show()
        fig9.show()
        fig10.show()

    else:
        print("\n❌ Failed to fetch data. Please check:")
        print("  1. Internet connection")
        print("  2. yfinance is up to date: pip install --upgrade yfinance")
        print("  3. Symbol 'SBIN.NS' is correct")

Fetching SBIN.NS stock data...
Data fetched: 251 rows from 2024-10-07 to 2025-10-06
Columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Adding technical indicators...

Total columns: 52
Total rows after cleaning: 52

LATEST SBIN.NS STOCK DATA WITH TECHNICAL INDICATORS

Date: 2025-10-06

--- PRICE DATA ---
Close Price: ₹874.05
Open: ₹868.00 | High: ₹875.30 | Low: ₹862.80
Volume: 7,819,450

--- TREND INDICATORS ---
SMA 20: ₹851.36 | SMA 50: ₹826.63
EMA 20: ₹853.43 | EMA 50: ₹834.32
MACD: 13.66 | Signal: 13.02 | Hist: 0.64
ADX: 38.04

--- MOMENTUM INDICATORS ---
RSI (14): 67.85
Stochastic K: 86.97 | D: 78.98
DeMarker: 0.76

--- VOLATILITY INDICATORS ---
ATR (14): ₹11.25
Bollinger Upper: ₹892.25 | Middle: ₹851.36 | Lower: ₹810.48
Std Dev (20): ₹20.97

--- VOLUME INDICATORS ---
OBV: 98,372,402
ADI: -94,730,777.69

--- SUPPORT & RESISTANCE ---
Pivot: ₹870.72
Resistance: R1=₹878.63 | R2=₹883.22 | R3=₹891.13
Support: S1=₹866.13 | S2=₹858.22 | S3=₹853.63
VWAP: ₹787.